In [18]:
from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_euler('zyx', [
[40, 0, 0],
[-40, 0, 0],
[0, 0, 0]],degrees=True)
r=r.as_matrix()
print(np.array([[-0.05],[-1],[0.25]]))
# T_12=np.concatenate(r[0],np.array([[-0.05],[-1],[0.25]]))
# T_12=list(r[0]).append([[-0.05],[-1],[0.25]])
# print(T_12)
# T= [9np.identity(4)  [np.hstack((r[0], [[-0.05],[-1],[0.25]])),[0, 0, 0, 1]]]
T_01 = np.identity((4))
T_01[:3,:3] = np.copy(r[0])
T_01[:3,3] = np.copy(np.array([[-0.05],[-1],[0.25]]).T)

T_02 = np.identity((4))
T_02[:3,:3] = np.copy(r[1])
T_02[:3,3] = np.copy(np.array([[-0.05],[1],[0.25]]).T)

T_03 = np.identity((4))
T_03[:3,:3] = np.copy(r[2])
T_03[:3,3] = np.copy(np.array([[-0.02],[0],[0.05]]).T)

T_10=np.linalg.inv(T_01);
T_20=np.linalg.inv(T_02);
T_30=np.linalg.inv(T_03);
T_21=np.dot(np.linalg.inv(T_02),T_01);
T_31=np.dot(np.linalg.inv(T_03),T_01);
T_12=np.linalg.inv(T_21);
T_23=np.dot(np.linalg.inv(T_02),T_03);
T_13=np.linalg.inv(T_31);
T_32=np.linalg.inv(T_23);

# T_12 = np.identity((4))
# T_12[:3,:3] = np.copy(r[3])
# T_12[:3,3] = np.copy(np.array([[-2],[0],[0]]).T)

# T_13 = np.identity((4))
# T_13[:3,:3] = np.copy(r[4])
# T_13[:3,3] = np.copy(np.array([[-1],[-0.03],[0.25]]).T)

# T_23 = np.identity((4))
# T_23[:3,:3] = np.copy(r[5])
# T_23[:3,3] = np.copy(np.array([[1],[-0.03],[0.25]]).T)


# T_10 = np.identity((4))
# T_10[:3,:3] = np.copy(r[6])
# T_10[:3,3] = np.copy(np.array([[0.05],[1],[-0.25]]).T)

# T_20 = np.identity((4))
# T_20[:3,:3] = np.copy(r[7])
# T_20[:3,3] = np.copy(np.array([[0.05],[-1],[-0.25]]).T)

# T_21 = np.identity((4))
# T_21[:3,:3] = np.copy(r[8])
# T_21[:3,3] = np.copy(np.array([[2],[0],[0]]).T)

# T_30 = np.identity((4))
# T_30[:3,:3] = np.copy(r[9])
# T_30[:3,3] = np.copy(np.array([[0.02],[0],[-0.5]]).T)

# T_31 = np.identity((4))
# T_31[:3,:3] = np.copy(r[10])
# T_31[:3,3] = np.copy(np.array([[1],[0.03],[-0.025]]).T)

# T_32 = np.identity((4))
# T_32[:3,:3] = np.copy(r[11])
# T_32[:3,3] = np.copy(np.array([[-1],[0.03],[0.25]]).T)


T=[[np.identity(4) ,T_01, T_02 ,T_03] , [T_10, np.identity(4), T_12, T_13], [T_20 ,T_21 ,np.identity(4), T_23],[T_30 ,T_31, T_32 ,np.identity(4)]]


print(T_21)

[[-0.05]
 [-1.  ]
 [ 0.25]]
[[ 0.17364818 -0.98480775  0.          1.28557522]
 [ 0.98480775  0.17364818  0.         -1.53208889]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [19]:
def transformation(points, rotation_matrix):
    new_points= np.empty([points.shape[0], 3])
    for n in range(0,points.shape[0]):
        point=np.array([points[n,0],points[n,1],points[n,2],1])
        new_points[n,:]=rotation_matrix[:3,:]@point
    return new_points


def best_fit_transform(A, B):
    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m+1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t

def nearest_neighbor(src, dst):
    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=1000, tolerance=0.0001):
    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            break
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

In [20]:
from itertools import permutations         
        
def GetSensorPathSequences(*args):
    # N - number of sensors
    # L - path length (1 to N-1)
    # T - target sensor (1 to N-1)
    n_arg= len(args)
    if n_arg==3:
        N=args[0]
        L=args[1]
        T=args[2]
    elif n_arg<3:
        N=args[0]
        L=args[1]
        T=3
    elif n_arg<2:
        N=args[0]
        L=2
    elif n_arg<1:
        N=5
    seqs=list(permutations(range(1,N)))
    zeros=list(np.zeros(len(seqs[0]),int))
    zeros=tuple(zeros)
    seqs=[zeros]+seqs
    seqs=np.array(seqs)
    matches=[]
    for x in seqs:
        if x[L-1]==T:
            matches.append((x[:L]))
    matches = np.unique(matches, axis=0)
#     print(matches)
    return matches
    

    
GetSensorPathSequences(6,2,4)

array([[1, 4],
       [2, 4],
       [3, 4],
       [5, 4]])

In [97]:
def DerivedTransformation(tT,n,m,k,L,N):
#     print(tT)
    if n!=0:
        return -1
    k_min=2
#     k_max=1
#     if L!=0:
#         if L==2:
#             k_min=1
#             k_max=(N-2)-1
#         elif L==3:
#             k_min=N-2
#             K_max=(N-2)*(N-3)-1
#         else:
#             for r in range(2,L):
#                 k_min=k_min*(N-r)
#             for r in range(2,L+1):
#                 print(r)
#                 k_max=k_max*(N-r)
#             k_max=k_max-1
#     print("kmin:",k_min, "kmax:", k_max)
#     if k >= k_min and k<=N-2 :
    seqs=GetSensorPathSequences(N,L,m)
#     else:
#         return -1
    idx=k-k_min
    seqk=seqs[idx]
    seqk=[0,*seqk]
#     print(k,seqk, len(seqk))
    T=np.identity(4)
    for t in range(0,len(seqk)-1):
        T=np.dot(T,tT[seqk[t]][seqk[t+1]])
    Tp=T
    print("seq:",seqk,"\n tT[0][seqk[0]]:\n",tT[0][seqk[0]],"\ntT[seqk[0]][seqk[1]]:\n" ,tT[seqk[0]][seqk[1]], "\nTp:\n",Tp)
#     print(Tp)
    return Tp
    
    

In [98]:
def DerivedTransformationSimples(tT,m,k,N):
    if k>=1 and k<=N-2:
        seqs=GetSensorPathSequences(N,2,m)
    else:
        return -1
    seqk=seqs[k-1]
    Tp=np.dot(tT[0][seqk[0]],tT[seqk[0]][seqk[1]])
    print("seq:",seqk,"\n tT[0][seqk[0]]:\n",tT[0][seqk[0]],"\ntT[seqk[0]][seqk[1]]:\n" ,tT[seqk[0]][seqk[1]], "\nTp:\n",Tp)
    return Tp

In [99]:
def MeanTransformation(Tp):
    k=3
    tt=np.zeros([1,3],dtype=np.float64)
    Tt=np.zeros([3,3],dtype=np.float64)
    for i in range(0,k):
        Tt+=np.array(Tp[i][:3,:3])
        tt+=np.array(Tp[i][:3,3]).T
#     print(Tt,tt)
    
    t_average=1/k*tt
    u,s,vh=np.linalg.svd(Tt)
    R_average=np.dot(u, vh)

    T_average = np.identity(4)
    T_average[:3, :3] = R_average
    T_average[:3, 3] = t_average
    
    return T_average

In [100]:
# def MultiPairWiseCalibration
N=4
Tp=[]
ht=[]
for m in range(1,N):
#     print(m)
    Tp=[]
    Tp.append(T[0][m])
    print(T[0][m])
    for k in range(1,N-1):
        print(k)
        Tp.append(DerivedTransformationSimples(T,m,k,N))
        Tp.append(DerivedTransformation(T,0,m,k,3,N))
    print(Tp)

    ht.append(MeanTransformation(Tp))
# print (ht)

for i in range(0,3):
    print( "ht matrix: \n", ht[i],"\n Direct matrix (",0,",",i+1,"): \n",T[0][i+1])

# for i in range(0,len(ht)):
#     m = objectPoints[0].shape[1]
#     src = np.ones((m+1,objectPoints[0].shape[0]))
#     dst_ht = np.ones((m+1,objectPoints[0].shape[0]))
#     src[:m,:] = np.copy(objectPoints[0].T)
#     dst_normal=np.ones((m+1,objectPoints[0].shape[0]))
#     dst_normal[:m,:]=np.copy(objectPoints[i+1].T)
# #     dst[:m,:] = np.copy(B.T)
#     dst_ht=np.dot(ht[i], src)
# #     print(src,dst)
#     distances, indices = nearest_neighbor(dst_normal[:m,:].T, dst_ht[:m,:].T)
#     distances_2,indices  = nearest_neighbor(dst_ht[:m,:].T, dst_normal[:m,:].T)
#     print("From 0 to", i+1, ":",np.mean(distances), ",",np.mean(distances_2), "Normal distance :", distances_vec[0][i+1])

[[ 0.76604444 -0.64278761  0.         -0.05      ]
 [ 0.64278761  0.76604444  0.         -1.        ]
 [ 0.          0.          1.          0.25      ]
 [ 0.          0.          0.          1.        ]]
1
seq: [2 1] 
 tT[0][seqk[0]]:
 [[ 0.76604444  0.64278761  0.         -0.05      ]
 [-0.64278761  0.76604444 -0.          1.        ]
 [-0.          0.          1.          0.25      ]
 [ 0.          0.          0.          1.        ]] 
tT[seqk[0]][seqk[1]]:
 [[ 0.17364818 -0.98480775  0.          1.28557522]
 [ 0.98480775  0.17364818  0.         -1.53208889]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]] 
Tp:
 [[ 0.76604444 -0.64278761  0.         -0.05      ]
 [ 0.64278761  0.76604444  0.         -1.        ]
 [ 0.          0.          1.          0.25      ]
 [ 0.          0.          0.          1.        ]]
seq: [0, 3, 2, 1] 
 tT[0][seqk[0]]:
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]] 
tT[seqk[0]][seqk[1]